# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. Write the pseudo code}
\\
&\text{2. }
\end{align}


#### Remarks
STA can be solve with link flows. But if you do that, you will not acces the path flows.

It can also be solve using paths flow. 
This is hard, because finding every path in a network in a NP-complete problem.
If you assume that you already know all the path in the network, you need a big memory.

The tradeoff memory/running time has been, for years, done with low memory computers.
Now, because we have HPC, we can use more memory. If we can now have more memory, we still can store all the possible path of a network. A idea would be to store only the path used in the network.

To know the path used in the network, one can compute few steps of the Frank Wolfe algorithm.

The running time bottle neck in the Frank Wolfe algorithm is computing the shortest path algorithm only with links flows.
But if we have path flow, this could be way easier.
This code is a trial to do such an algorithm. Test will be done once the proof of concept will be achieved.

In [ ]:
debug = True

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [ ]:
import numpy as np
import scipy.sparse

# Load the data

### We clean the data

In [ ]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a0, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1,
    and a0, ..., a4 are the coeficient to calculate the travel time of one link as a function
    of the flow on the link: t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4

One can add some checks here to be sure that demand and graph respect this format!

Some demand and graph data can be found on:
    https://github.com/bstabler/TransportationNetworks
    Here one need to perprocess the data before using this code
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    # WE SHOULD ALSO CHECK THAT EVERY DEMAND IS WELL DEFINED: >0
    return graph, demand

def load_network(network):
    graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
    demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)
    graph, demand = cleaning_input(graph, demand)
    if debug:
        print("The network " + network + " has been charged. The network characteristics are:")
        nb_links = int(np.max(graph[:,0])+1)
        nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
        nb_ods = int(demand.shape[0])
        print("\t nb nodes = " + str(nb_nodes))
        print("\t nb links = " + str(nb_links))
        print("\t nb ods = " + str(nb_ods))
    return graph, demand

debug_local = True
if debug_local:
    network_name = 'data/SiouxFalls_bis'
    graph, demand = load_network(network_name)

In [ ]:
# link, a, b, B, power, c, fft

Then, we define the function which gives the travel time as a function of the flow

In [ ]:

"""
We define the travel time of each link as a function of the flow on each link.
This function is given by the network topology (in the graph file) and by the capacity of each link.
One row of the graph is:
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4]
Each row correspond to one link.
The travel time of the link is t = fft * (1 + B * (f/c)^power)
"""
def travel_time(graph, f):
    # here we need to have the same indexation for graph and f. That why we need to store graph in a dictionnary
    # this could be changed
    return graph[:,6] * (1 + graph[:,3]*(f/graph[:,5])**graph[:,4])

## 2. We compute the all or nothing flow allocation

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [ ]:
"""
We store the travel time of each link in a sparce adjacency matrix G.
Given a flow allocation f and a capacities constraints c, we compute 
the travel time of each link and store it in G.

This allow us after to us the Disjtra's algorithm of the class scipy.sparse.csgraph
"""

def update_travel_time_from_tt(graph, tt, nb_nodes):
    G = np.zeros(shape=(nb_nodes,nb_nodes))
    G = scipy.sparse.lil_matrix(G)
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    G = G.tocsr()
    return G

def update_travel_time_from_flow(graph, f, nb_nodes):
    return update_travel_time_from_tt(graph,travel_time(graph, f), nb_nodes)

debug_local = False
if debug_local:
    graph, _ = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes) # , [1,1,-1,1,1]))
    print(G)

In [ ]:
from scipy.sparse.csgraph import dijkstra

In [ ]:
"""
Because the classic Frank Wolf algorithm does not take into account the paths,
we need to store the path in memory. We do it by using Object and a dictionnary 
called paths_used.

This idea is that we do not want to compute all the possible paths of the network
before the beginning of the algorithm. So we compute the paths used during the algorithm,
and we store them in memory.

We build at the same time the incidence matrix.

CAREFUL. We use the number of links of the graph in the calcul of the hash of every path.
This variable should be a parameter.
"""

# we need this class to know if we already know the path in the all or nothing part
# the number of links of the network is needed to compute the hashcode of the path
class path:
    __nb_links = -1
    def set_nb_links(nb_l):
        path.__nb_links = nb_l
    def __init__(self,links):
        self.links = links
        self.__flow = 0
        if path.__nb_links == -1:
            raise Exception('The number of links as not be defined inside the path class. We need it for defining the hash of a link. Please use the function path.set_nb_links(nb_links) to define it')
    def set_flow(self, flow):
        self.__flow = flow
    def get_flow(self,):
        return self.__flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    # I am not sure about the hash table structure in Python. I am used to Java.
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(path.__nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.__flow) + " is on " + str(self.links) 

debug_local = False
if debug_local:
    graph, _ = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    path.set_nb_links(nb_links)
    paths_used = {}
    print(paths_used)
    p = path([0, 1, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 1, 1, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 0, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)

Now let's compute the all or nothing allocation

In [ ]:
"""
This cell is the main point of the entire solver.
It is the all or nothing allocation.

"""


# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
def build_graph_adjacency(graph):
    graph_dict = {}
    for i in range(graph.shape[0]):
        try: 
            graph_dict[int(graph[i][1])]
        except:
            graph_dict[int(graph[i][1])] = {}
        graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    return graph_dict

# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
def build_demand_dict(demand):
    demand_dict = {}
    for i in range(demand.shape[0]):
        try: 
            demand_dict[int(demand[i][0])]
        except:
            demand_dict[int(demand[i][0])] = {}
        demand_dict[int(demand[i][0])][int(demand[i][1])] = i
    return demand_dict

def put_flow_on_short_path(faon, o_tmp, d_tmp, flow_tmp, return_predecessors, graph_dict, paths_used, paths_used_tmp, k, i):
    node_tmp = d_tmp
    links = []
    # using the dijkstra, we build the fastest path and we put the flow on it.
    while node_tmp != o_tmp:
        if debug_local:
            print(o_tmp)
            print(node_tmp)
        node_tmp_d = return_predecessors[o_tmp][node_tmp]
        # Here we need the graph_dict to recover the link id from the nodes id.
        link_tmp = graph_dict[node_tmp_d][node_tmp]
        # we recover the path from the predecessor of the 
        links.insert(0, link_tmp)
        faon[link_tmp] += flow_tmp
        node_tmp = node_tmp_d

    p = path(links)
    p.set_flow(flow_tmp)


    if debug_local:
        print(p)
    # If we do not already know p, we add it to the delta matrix (here a dict)
    if not hash(p) in paths_used:
        if debug_local:
            print(k)
        # we add p and his index
        paths_used[hash(p)] = (p, k)
        paths_used_tmp[hash(p)] = (p, k)
        k = k+1
    else:
        # we recover the index of p using the paths_used dict
        paths_used_tmp[hash(p)] = (p, paths_used[hash(p)][1])
    return faon, paths_used, paths_used_tmp, k


# computing the all or nothing flow
def all_or_nothing_dikjstra(demand, G, graph_dict, paths_used, k):
    # k is the next index to give to a new path.
    debug_local = True
    # paths_used_tmp is the set of the path in the all or nothing allocation of this step
    paths_used_tmp = {}
    # faon is the flow allocation of the all or nothing allocation
    nb_links = G.nnz
    faon = np.zeros(nb_links)
    
    # using scipy to compute dijkstra"
   
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    if debug_local:
        print(return_predecessors)
        
    # for every origin destination pairs (i.e. one line of the demand file)
    nb_ods = int(demand.shape[0])
    for i in range(nb_ods):
        # we compute the shortest path and add the demand on it.
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]
        faon, paths_used, paths_used_tmp, k = put_flow_on_short_path(faon, o_tmp, d_tmp, flow_tmp, return_predecessors, graph_dict, paths_used, paths_used_tmp, k, i)

    return faon, paths_used_tmp, paths_used, k

We define the line search

In [ ]:
debug_local = True
if debug_local:
    # we load the network
    graph, demand = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    print(nb_links)
    path.set_nb_links(nb_links)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes)
    graph_dict = build_graph_adjacency(graph)
    paths_used = {}
    k = 0 
    _, _ , pp, k = all_or_nothing_dikjstra(demand, G, graph_dict, paths_used, k)
    for p in pp.values():
        print(p[0])
    all_or_nothing_dikjstra(demand, G, graph_dict, paths_used, k)
    for p in paths_used.values():
        print(p[0])

In [ ]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.

The function line search does a 1D line search.
"""

def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,6] * (f + graph[:,3]*(f/(graph[:,4]+1))*(f/graph[:,5])**graph[:,4])
    return np.sum(pot_tmp)


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [ ]:
def build_network(graph):
    nb_links = int(np.max(graph[:,0])+1)
    path.set_nb_links(nb_links)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes)
    graph_dict = build_graph_adjacency(graph)
    return nb_links, nb_nodes, G, graph_dict
    
def initialization_FW(demand, G, graph_dict, all_paths_used, k, graph, nb_nodes):
    debug_local = False
    
    f, paths_used_for_this_iter, all_paths_used, k = all_or_nothing_dikjstra(demand, G, graph_dict, all_paths_used, k)
    if debug_local:
        print(G)
        print(f)
    G = update_travel_time_from_flow(graph, f, nb_nodes)
    path_flow_matrix = np.zeros(k)
    for val in paths_used_for_this_iter.values():
        path_flow_matrix[val[1]] = val[0].get_flow()

    if debug:
        print("Test of initialization_FW")
        print(f)
        for p in paths_used_for_this_iter.values():
            print(p[0])
        print(path_flow_matrix)
    return f, paths_used_for_this_iter, all_paths_used, k, G, path_flow_matrix

def iteration_FW(demand, G, graph_dict, all_paths_used, k, graph, f, nb_nodes, path_flow_matrix, i):
    # WE COMPUTE THE ALL OR NOTHING ALGORITHM
    faon, paths_used_for_this_iter, all_paths_used, k = all_or_nothing_dikjstra(demand, G, graph_dict, all_paths_used, k)
    
    # we find the better convex combinaison of f and faon
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    # HERE WE SHOULD BE CAREFUL IN THE CASE WHERE WE HAVE THE CAPACITY CONSTRAINTS
    # THE GRADIENT OF THE FUNCTION IS NOT THE SHORTEST PATH
    # WE SHOULD REMOVE THE PATH THAT SATURATED THE LINKS
    # AND THEN COMPUTE THE SOLUTION WITHOUT THE SATURATION, AND WITHOUT THE 
    # CORRESPONDING DEMAND
    if s==0:
        update_flow = True
        
        demand_tmp = demand.copy()
        nb_links = int(np.max(graph[:,0])+1)
        _, delta, route2od = output_FW(all_paths_used, nb_links, graph, f, demand)
        # WE COMPUTE THE ALL OR NOTHING ALGORITHM
        faon, paths_used_for_this_iter, all_paths_used, k = all_or_nothing_dikjstra(demand_tmp, G, graph_dict, all_paths_used, k)

        # I add the flow of the saturated path to faon 
        for path_c in path_at_capacity:
            for j in np.where(delta[path_c].toarray()[0]==1):
                for jj in j:
                    faon[jj] += path_flow_matrix[path_c]

        # we find the better convex combinaison of f and faon
        s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
        if debug:
            print(s)
            print(potential(graph, (1. - s) * f + s * faon))
        for path_c in path_at_capacity:
            path_flow_matrix[path_c] = path_flow_matrix[path_c] /(1-s) 
    else:
        update_flow = False
    f = (1. - s) * f + s * faon
    G = update_travel_time_from_flow(graph, f, nb_nodes)

    # we multiply the previous path flow matrix by the coeficient of the line search
    path_flow_matrix = (1-s) * path_flow_matrix
    # we add the new path at the end of the path flow matrix.
    path_flow_matrix = np.append(path_flow_matrix, np.zeros(len(all_paths_used)-path_flow_matrix.shape[0]))
    # we add the all or nothing path flow (mulitply by the coeficient of the line search) to the path flow matrix.
    for val in paths_used_for_this_iter.values():
        # val[1] is the index of the path, val[0] is the path object
        path_flow_matrix[val[1]] += s * val[0].get_flow()

    if debug and i % (nb_iter / 10) == 0:
        print("Iteration: " + str(i))
        print("demand = " + str(demand))
        print("s: " + str(s))
        print("h: " +str(path_flow_matrix))
        print("f: " + str(f))
        print("The paths used at this iteration are:")
        for p in paths_used_for_this_iter.values():
            print(p[0])
    return path_flow_matrix, f, G, all_paths_used, k, s

def output_FW(all_paths_used, nb_links, graph, f, demand):
    # MAYBE I SHOULD SPLIT THE CODE HERE TO MAKE EASIER TEST
    # At the end we do some work to return a proper output
    nb_paths = len(all_paths_used.keys())
    delta = np.zeros(shape=(nb_paths, nb_links)) # nb_links should be a parameter
    route2od = [0 for _ in range(nb_paths)]# np.zeros(shape=nb_paths)
    delta = scipy.sparse.lil_matrix(delta)
    tt_f = np.array(travel_time(graph, f))
    
    demand_dict = build_demand_dict(demand)

    for p in all_paths_used.values():
        # here I can built route2od matrix at the same time
        try:
            links_tmp = p[0].links
            for l in links_tmp:
                delta[p[1],l] = 1
            route2od[p[1]] = demand_dict[int(graph[int(links_tmp[0])][1])][int(graph[int(links_tmp[-1])][2])]
        except:
            ;
    delta = delta.tocsr()
    # I should built the route2od matrix here,
    # route2od should be build from the all_paths_used_dict
    return tt_f, delta, route2od

see_results = False
def Frank_Wolf_solver(graph, demand_bis, eps, nb_iter):
    ######### FIRST, WE INITIALIZE THE ALGORITHM #########
    # We initialize the number of paths_used to 0, 
    k = 0
    all_paths_used = {}
    
    # we built the network as a matrix from the graph file
    nb_links, nb_nodes, G, graph_dict = build_network(graph)
    
    # The initialization step: we put all the demand on the fastest free flow travel time paths.
    f, paths_used_for_this_iter, all_paths_used, k, G, path_flow_matrix = initialization_FW(demand_bis, G, graph_dict, all_paths_used, k, graph, nb_nodes)

    ######### THEN, I RUN THE ITERATION OF THE FRANK-WOLF ALGORITHM #########
    for i in range(nb_iter):
        path_flow_matrix, f, G, all_paths_used, k, s = iteration_FW(demand_bis, G, graph_dict, all_paths_used, k, graph, f, nb_nodes, path_flow_matrix, i)
        if s < eps:
            if see_results:
                print(i, s)
            break
        if see_results and (i % (nb_iter/20)==0):
            print("Iteration " + str(i) + ", error = " + str(s))
    if debug:
        print(path_flow_matrix)
       
    ######### FINALLY, I WORK ON THE OUTPUT TO RETURN #########
    tt_f, delta, route2od = output_FW(all_paths_used, nb_links, graph, f, demand_bis)
    return path_flow_matrix, tt_f, delta.toarray(), route2od


In [ ]:
def check_wardrop(j, demand, route2od, delta, path_flow_matrix, tt_f):
    f = delta.T @ path_flow_matrix
    tt_p = delta @ tt_f
    od = j
    tab = []
    for i in range(len(route2od)):
        if route2od[i] == od:
            tab.append(i)
    print(od)
    print(tab)
    print(tt_p[tab])

def od2route(route2od):
    od2route = {}
    for i in range(len(route2od)):
        if route2od[i] not in od2route.keys():
            od2route[route2od[i]] = set()
        od2route[route2od[i]].add(i)
    print(route2od)
    return od2route

In [ ]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Siou2 = 'data/SiouxFalls_bis'
Brae = 'data/braess'

debug=True

network_name = Siou2
eps=1e-8
nb_iter = 1000
graph, demand = load_network(network_name)
if network_name == Brae:
    demand[0][2] = 10

debug = False
see_results = True

path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver(graph, demand, eps, nb_iter) #, [11,11,2,11,11])

if debug:
    nb_paths = len(path_flow_matrix)
    print(path_flow_matrix)
    print(nb_paths)
    print(tt_f)
    print(delta)
    print(delta @ tt_f)
    print(route2od)
    print(delta.shape)
    print(od2route(route2od))
    for od in od2route(route2od).keys():
        print("check wardrop for the od: " + str(od))
        check_wardrop(od, demand, route2od, delta, path_flow_matrix, tt_f)


In [ ]:
print(tt_f)
f = delta.T @ path_flow_matrix
print(f)

In [ ]:
solution = np.loadtxt(network_name + '_result.csv', delimiter=',', skiprows=1)
# print(solution[:,1] - f)
print(f[27])
print(f[42])

In [ ]:
print(solution[:,2] - tt_f)

In [ ]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Siou2 = 'data/SiouxFalls_bis'
Brae = 'data/braess'

debug=True

network_name = Siou2
eps=1e-8
nb_iter = 1000
graph, demand = load_network(network_name)

u = np.ones(int(np.max(graph[:,0])+1))
u = u * 50000
u[27] = 20000
u[42] = 20000
debug = False
see_results = True
float_appro = 1e-3

path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver(graph, demand, eps, nb_iter, u) #, [11,11,2,11,11])

if debug:
    nb_paths = len(path_flow_matrix)
    print(path_flow_matrix)
    print(nb_paths)
    print(tt_f)
    print(delta)
    print(delta @ tt_f)
    print(route2od)
    print(delta.shape)
    print(od2route(route2od))
    for od in od2route(route2od).keys():
        print("check wardrop for the od: " + str(od))
        check_wardrop(od, demand, route2od, delta, path_flow_matrix, tt_f)



In [ ]:
print((delta.T @ path_flow_matrix)[42])







# New try





In [3]:
import numpy as np
network = 'data/SiouxFalls_bis'
# network = 'data/Braess_bis'
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)
t0 = graph[:,6]
B = graph[:,3]
c = graph[:,5]
power = graph[:,4]
nb_links = graph.shape[0]
del graph

# here we shoud read delta and route2od.

In [4]:
f = np.zeros(nb_links)
tt = t0 * (1 + B*(f/c)**power)
print(tt)

[ 6.  4.  6.  5.  4.  4.  4.  4.  2.  6.  2.  4.  5.  5.  4.  2.  3.  2.
  2.  3. 10.  5.  5. 10.  3.  3.  5.  6.  4.  8.  6.  5.  6.  4.  4.  6.
  3.  3.  4.  4.  5.  4.  6.  5.  3.  3.  5.  4.  2.  3.  8.  2.  2.  2.
  3.  4.  3.  2.  4.  4.  4.  6.  5.  6.  2.  3.  3.  5.  2.  4.  4.  4.
  2.  4.  3.  2.]


In [9]:
solution = np.loadtxt(network + '_result.csv', delimiter=',', skiprows=1)

tt = t0 * (1 + B*(solution[:,1]/c)**power)

print(tt)
print(solution[:,2])

[ 6.9   4.6   6.9   5.75  4.6   4.6   4.6   4.6   2.3   6.9   2.3   4.6
  5.75  5.75  4.6   2.3   3.45  2.3   2.3   3.45 11.5   5.75  5.75 11.5
  3.45  3.45  5.75  6.9   4.6   9.2   6.9   5.75  6.9   4.6   4.6   6.9
  3.45  3.45  4.6   4.6   5.75  4.6   6.9   5.75  3.45  3.45  5.75  4.6
  2.3   3.45  9.2   2.3   2.3   2.3   3.45  4.6   3.45  2.3   4.6   4.6
  4.6   6.9   5.75  6.9   2.3   3.45  3.45  5.75  2.3   4.6   4.6   4.6
  2.3   4.6   3.45  2.3 ]
[ 6.00081624  4.00869075  6.00083412  6.57359826  4.00858665  4.26940183
  4.02017916  4.27126776  2.31537411  7.13330048  2.31707223  9.99822521
  9.65131071  6.59951767 10.02070256 14.690955    5.55216038  2.06222569
 14.82415952  5.50141296 15.17470751 10.72947353  9.67015456 15.0378695
  5.68253305  5.71724339 12.40568945 13.72237028 20.08480998 16.30801715
  7.22302456 12.20325453 13.59022763 13.69128569  4.01979049 13.73515565
  3.02279654  3.02347967 17.66100772 13.84264505 12.23433913  9.07934431
 13.81156045 12.37460486  4.3262

In [ ]:
def path_all_or_nothing(demand, tt, delta, od2route):
    """
    All or nothing allocation using the incidence matrix.
    """
    np_paths = delta.shape[1]
    hp = np.zeros(np_paths)
    tp = delta.T @ tt
    for od in range(demand.shape[0]):
        # pour l'instant que une od
        tp_tmp = tp[np.nonzero(np.array(od2route) == od)]
        shortest_path = np.argmin(tp_tmp)
        hp[shortest_path] = demand[od][2]
    return hp

def potential(f, t0, B, c, power):
    """
    This routing is useful for doing a line search.
    It computes the potential at flow assignment f.
    """
    return np.sum(t0*(f + B * (f/(power+1))*((f/c)**power)))


def line_search(f, res=20):
    """
    on a grid of 2^res points bw 0 and 1, find global minimum
    of continuous convex function
    here we do a bisection
    """
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

In [ ]:
import Frank_Wolf_solver
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network_name = Brae

eps=1e-8
nb_iter = 1000
graph, demand = Frank_Wolf_solver.load_network(network_name)
if network_name == Brae:
    demand[0][2] = 10
path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver.Frank_Wolf_solver(graph, demand, eps, nb_iter)
delta = delta.T
print(path_flow_matrix)
print(tt_f)
print(delta)
print(delta.T @ tt_f)

In [ ]:
test_path = np.array([3.5, 5.6, 3.2, 3.7, 5.8])
test_route = np.array([0, 1, 0, 0, 1])
print(test_route)
print(test_path)
indices = np.nonzero(test_route == 0)
print(indices)
print(test_path[indices])
print(np.argmin(test_path[indices]))

In [ ]:
# an first test
hp = path_all_or_nothing(demand, tt, delta, route2od)
f = delta @ hp

i = 0
nb_iters = 100
while i < nb_iters:
    tt = t0 * (1 + B*(f/c)**power)
    hpaon = path_all_or_nothing(demand, tt, delta, route2od)
    faon = delta @ hpaon
    s = line_search(lambda a: potential((1. - a) * f + a * faon, t0, B, c, power))
    hp = s*hpaon + (1-s)*hp
    f = delta @ hp
print(f)

In [ ]:
f = np.zeros(nb_links)
tt = t0 * (1 + B*(f/c)**power)
f = all_or_nothing(demand, tt, delta, od2route)

In [ ]:
# One possibility
# TO DO: WRITE all_or_nothing and line_search
i = 0
while i<10:
    faon = all_or_nothing(g, tt, demand)
    f = line_search(f, faon, lambda a: np.sum(t0*(a + B * (a/(power+1))*((a/c)**power))))
    tt = t0 * (1 + B*(f/c)**power)
    i = i+1

In [ ]:
# a last one
all_or_nothing(g, tt)
    tp = delta.T @ tt
    p = np.argmin(tp)

In [ ]:
all_or_nothing(g, tt)
    

In [ ]:
help(potential)